In [2]:
import importlib

import pandas as pd
import numpy as np

importlib.import_module("helpers")

from helpers import convert_elapsed_time, extract_num, build_era
from helpers import add_prefix, listing_type, roof_description

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [3]:
df = pd.read_json("../scraping/ams_sale.json")

In [4]:
df.shape

(4091, 78)

In [5]:
df.head()

,address,postcode,city,OVE-Vraagprijs,OVE-Vraagprijs per m²,OVE-Aangeboden sinds,OVE-Status,OVE-Aanvaarding,BOU-Soort woonhuis,BOU-Soort bouw,BOU-Bouwjaar,BOU-Soort dak,OPP-,OPP-Perceel,OPP-Inhoud,IND-Aantal kamers,IND-Aantal badkamers,IND-Badkamervoorzieningen,IND-Aantal woonlagen,IND-Voorzieningen,ENE-Energielabel,ENE-Isolatie,ENE-Verwarming,ENE-Warm water,ENE-Cv-ketel,KAD-,BUI-Tuin,BUI-Balkon/dakterras,GAR-Soort garage,GAR-Capaciteit,GAR-Voorzieningen,PAR-Soort parkeergelegenheid,BUI-Ligging,BUI-Voortuin,BUI-Ligging tuin,OVE-Servicekosten,BOU-Specifiek,BUI-Zonneterras,BER-Schuur/berging,BER-Voorzieningen,BUI-Achtertuin,BER-Isolatie,BOU-Keurmerken,BUI-Patio/atrium,OVE-Bijdrage VvE,BOU-Soort appartement,BOU-Bouwperiode,IND-Gelegen op,VVE-Inschrijving KvK,VVE-Jaarlijkse vergadering,VVE-Periodieke bijdrage,VVE-Reservefonds aanwezig,VVE-Onderhoudsplan,VVE-Opstalverzekering,GAR-Isolatie,BOU-Toegankelijkheid,OVE-Oorspronkelijke vraagprijs,ENE-Voorlopig energielabel,OVE-Huurprijs,OVE-Huurovereenkomst,BUI-Plaats,BUI-Zijtuin,OVE-Oorspronkelijke huurprijs,BOU-Soort overig aanbod,BED-Praktijkruimte,OVE-Koopmengvorm,BOU-Soort parkeergelegenheid,IND-Capaciteit,IND-Afmetingen,VEI-Prijs,VEI-Veilingperiode,VEI-Soort veiling,VEI-Veilingpartij,BED-Bedrijfsruimte,BED-Kantoorruimte,BED-Winkelruimte,IND-Perceel,BOU-Soort object
0,Buiksloterbreek 41,1034 XC,Amsterdam,€ 475.000 kosten koper,€ 5.220,26 september 2020,Beschikbaar,In overleg,"Eengezinswoning, geschakelde woning",Bestaande bouw,1994,Plat dak,,51 m²,379 m³,4 kamers (3 slaapkamers),1 badkamer en 1 apart toilet,Douche en toilet,2 woonlagen,"Buitenzonwering, schuifpui en TV kabel",B,Volledig geïsoleerd,Cv-ketel,Cv-ketel,Intergas HRE (gas gestookt combiketel uit 2010...,,Zonneterras,Dakterras aanwezig,Inpandig,1 auto,Stromend water,"Betaald parkeren, openbaar parkeren en parkeer...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arènpalmstraat 16,1104 DB,Amsterdam,€ 425.000 kosten koper,€ 3.400,17 september 2020,Beschikbaar,In overleg,"Eengezinswoning, hoekwoning",Bestaande bouw,2008,Plat dak,,98 m²,429 m³,6 kamers (4 slaapkamers),1 badkamer en 1 apart toilet,"Ligbad, douche en toilet",3 woonlagen,Mechanische ventilatie en TV kabel,A,Volledig geïsoleerd,Cv-ketel en gedeeltelijke vloerverwarming,Cv-ketel,Gas gestookt combiketel,,Voortuin,Dakterras aanwezig en balkon aanwezig,NaN,NaN,NaN,Op eigen terrein en openbaar parkeren,"Aan rustige weg, aan water, in woonwijk en vri...","36 m² (7 meter diep en 5,09 meter breed)",Gelegen op het zuidoosten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brigantijnkade 46,1086 VB,Amsterdam,€ 625.000 kosten koper,€ 4.340,18 september 2020,Beschikbaar,In overleg,"Villa, vrijstaande woning (waterwoning)",Bestaande bouw,2009,Plat dak bedekt met bitumineuze dakbedekking,,133 m²,519 m³,5 kamers (4 slaapkamers),2 badkamers,"Ligbad, 2 douches en 2 toiletten",3 woonlagen,Schuifpui,A,HR-glas en volledig geïsoleerd,Cv-ketel en gedeeltelijke vloerverwarming,Cv-ketel en elektrische boiler,"Nefit (gas gestookt combiketel uit 2017, eigen...",,Zonneterras,Dakterras aanwezig en frans balkon aanwezig,Parkeerkelder en parkeerplaats,NaN,NaN,Betaald parkeren,"Aan rustige weg, aan vaarwater, aan water, in ...",NaN,Gelegen op het zuidwesten,€ 14 per maand,Gedeeltelijk gestoffeerd,"33 m² (9,4 meter diep en 3,51 meter breed)",Inpandig,Elektra en stromend water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bosplaat 25,1025 AR,Amsterdam,€ 700.000 kosten koper,€ 5.833,19 september 2020,Beschikbaar,In overleg,"Bungalow, tussenwoning (semi-bungalow)",Bestaande bouw,1973,Plat dak,,160 m²,273 m³,4 kamers (3 slaapkamers),

In [6]:
df.columns

Index(['address', 'postcode', 'city', 'OVE-Vraagprijs',
       'OVE-Vraagprijs per m²', 'OVE-Aangeboden sinds', 'OVE-Status',
       'OVE-Aanvaarding', 'BOU-Soort woonhuis', 'BOU-Soort bouw',
       'BOU-Bouwjaar', 'BOU-Soort dak', 'OPP-', 'OPP-Perceel', 'OPP-Inhoud',
       'IND-Aantal kamers', 'IND-Aantal badkamers',
       'IND-Badkamervoorzieningen', 'IND-Aantal woonlagen',
       'IND-Voorzieningen', 'ENE-Energielabel', 'ENE-Isolatie',
       'ENE-Verwarming', 'ENE-Warm water', 'ENE-Cv-ketel', 'KAD-', 'BUI-Tuin',
       'BUI-Balkon/dakterras', 'GAR-Soort garage', 'GAR-Capaciteit',
       'GAR-Voorzieningen', 'PAR-Soort parkeergelegenheid', 'BUI-Ligging',
       'BUI-Voortuin', 'BUI-Ligging tuin', 'OVE-Servicekosten',
       'BOU-Specifiek', 'BUI-Zonneterras', 'BER-Schuur/berging',
       'BER-Voorzieningen', 'BUI-Achtertuin', 'BER-Isolatie', 'BOU-Keurmerken',
       'BUI-Patio/atrium', 'OVE-Bijdrage VvE', 'BOU-Soort appartement',
       'BOU-Bouwperiode', 'IND-Gelegen op', 'VVE-In

In [7]:
df.isna().sum()

address                              0
postcode                             0
city                                 0
OVE-Vraagprijs                       3
OVE-Vraagprijs per m²              119
OVE-Aangeboden sinds                 1
OVE-Status                           1
OVE-Aanvaarding                      6
BOU-Soort woonhuis                3451
BOU-Soort bouw                       5
BOU-Bouwjaar                       369
BOU-Soort dak                     1728
OPP-                               113
OPP-Perceel                       3531
OPP-Inhoud                         113
IND-Aantal kamers                  113
IND-Aantal badkamers               517
IND-Badkamervoorzieningen          913
IND-Aantal woonlagen               113
IND-Voorzieningen                 1186
ENE-Energielabel                  1005
ENE-Isolatie                      1025
ENE-Verwarming                     472
ENE-Warm water                     597
ENE-Cv-ketel                      1628
KAD-                     

In [8]:
df.dtypes

address                            object
postcode                           object
city                               object
OVE-Vraagprijs                     object
OVE-Vraagprijs per m²              object
OVE-Aangeboden sinds               object
OVE-Status                         object
OVE-Aanvaarding                    object
BOU-Soort woonhuis                 object
BOU-Soort bouw                     object
BOU-Bouwjaar                       object
BOU-Soort dak                      object
OPP-                               object
OPP-Perceel                        object
OPP-Inhoud                         object
IND-Aantal kamers                  object
IND-Aantal badkamers               object
IND-Badkamervoorzieningen          object
IND-Aantal woonlagen               object
IND-Voorzieningen                  object
ENE-Energielabel                   object
ENE-Isolatie                       object
ENE-Verwarming                     object
ENE-Warm water                    

# Make it happen

In [10]:
# Order of categories
col_trans = ["OVE", "BOU", "OPP",
             "IND", "ENE", "BUI",
             "GAR", "BER", "PAR", 
             "VVE", "KAD", "BED",
             "VEI"]

order = [x
         for i, col in enumerate(col_trans)
         for x in df.columns
         if x.startswith(col_trans[i])] 

df = df[["address", "postcode", "city"] + order]

In [11]:
# Rename columns
cols = {'OVE-Vraagprijs': 'asking_price',
        'OVE-Vraagprijs per m²': 'price_m2',
        'OVE-Aangeboden sinds': 'days_online',
        'OVE-Status': 'status',
        'OVE-Aanvaarding': 'acceptance',
        'BOU-Soort woonhuis': 'property_type',
        'BOU-Soort bouw': 'new_build',
        'BOU-Bouwjaar': 'build_year',
        'BOU-Soort dak': 'roof_type',
        'OPP-': 'opp',
        'OPP-Perceel': 'property_m2',
        'OPP-Inhoud': 'property_m3',
        'IND-Aantal kamers': 'num_rooms',
        'IND-Aantal badkamers': 'num_bathrooms',
        'IND-Badkamervoorzieningen': 'bathroom_features',
        'IND-Aantal woonlagen': 'floors',
        'IND-Voorzieningen':  'features',
        'ENE-Energielabel': 'energy_label',
        'ENE-Isolatie': 'isolation',
        'ENE-Verwarming': 'heating',
        'ENE-Warm water': 'hot_water',
        'ENE-Cv-ketel': 'boiler',
        'KAD-': 'kadaster',
        'BUI-Tuin': 'garden',
        'BUI-Balkon/dakterras': 'balcony',
        'GAR-Soort garage': 'garage_type',
        'GAR-Capaciteit': 'garage_size',
        'GAR-Voorzieningen': 'garage_features',
        'PAR-Soort parkeergelegenheid': 'parking',
        'BUI-Ligging': 'environment',
        'BUI-Voortuin': 'garden_front',
        'BUI-Ligging tuin': 'garden_orientation',
        'OVE-Servicekosten': 'service_fees_pm',
        'BOU-Specifiek': 'specials',
        'BUI-Zonneterras': 'terrace',
        'BER-Schuur/berging': 'storage_type',
        'BER-Voorzieningen': 'storage_features',
        'BUI-Achtertuin': 'garden_back',
        'BER-Isolatie': 'storage_isolation',
        'BOU-Keurmerken': 'certificates',
        'BUI-Patio/atrium': 'garden_patio',
        'OVE-Bijdrage VvE': 'vve_contribution',
        'BOU-Soort appartement': 'apartment_type',
        'BOU-Bouwperiode': 'build_era',
        'IND-Gelegen op': 'building_orientation',
        'VVE-Inschrijving KvK': 'vve_kvk',
        'VVE-Jaarlijkse vergadering': 'vve_am',
        'VVE-Periodieke bijdrage': 'vve_per_contr',
        'VVE-Reservefonds aanwezig': 'vve_reserve_fund',
        'VVE-Onderhoudsplan': 'vve_maintenance',
        'VVE-Opstalverzekering': 'vve_insurance',
        'GAR-Isolatie': 'garage_isolation',
        'BOU-Toegankelijkheid': 'accessibility',
        'OVE-Oorspronkelijke vraagprijs': 'asking_price_original',
        'ENE-Voorlopig energielabel': 'energy_label_temp',
        'OVE-Huurprijs': 'rent_price',
        'OVE-Huurovereenkomst': 'rental_agreement',
        'BUI-Plaats': 'garden_plaats',
        'BUI-Zijtuin': 'garden_side',
        'OVE-Oorspronkelijke huurprijs': 'rent_price_original',
        'BOU-Soort overig aanbod': 'prop_extra_type',
        'BED-Praktijkruimte': 'comp_practice',
        'OVE-Koopmengvorm': 'sale_type',
        'BOU-Soort parkeergelegenheid': 'parking_type',
        'IND-Capaciteit': 'parking_capacity',
        'IND-Afmetingen': 'prop_extra_dimensions',
        'VEI-Prijs': 'auction_price',
        'VEI-Veilingperiode': 'auction_period',
        'VEI-Soort veiling': 'auction_type',
        'VEI-Veilingpartij': 'auction_party',
        'BED-Bedrijfsruimte': 'company_space',
        'BED-Kantoorruimte': 'office_space',
        'BED-Winkelruimte': 'store_space',
        'IND-Perceel': 'ground_area',
        'BOU-Soort object': 'prop_build_area'}
df.rename(cols, axis=1, inplace=True)

In [12]:
# Initial drop of columns with little meaning
drop = ['status', 'acceptance', 'asking_price_original', 'rent_price',
        'rental_agreement', 'rent_price_original', 'sale_type', 'certificates',
        'accessibility', 'prop_extra_type', 'parking_type', 'prop_build_area', 
        'opp', 'parking_capacity', 'prop_extra_dimensions', 'ground_area', 
        'garden_plaats', 'garden_side', 'garage_size', 'garage_features',
        'garage_isolation', 'kadaster', 'comp_practice', 'company_space', 
        'office_space', 'store_space', 'auction_price', 'auction_period', 
        'auction_type', 'auction_party']
df.drop(columns=drop, inplace=True)

In [13]:
# Drop rows without asking price
df.dropna(subset=["asking_price"], inplace=True)

In [14]:
# Drop listings that are just garages and such
df = (df.drop(df[df["apartment_type"].isna() 
           & df["property_type"].isna()].index)
      .reset_index(drop=True))

In [15]:
# Convert columns in euro to numeric
euro = ["asking_price", "vve_contribution", 
        "service_fees_pm", "price_m2"]

for e in euro:
    df[e] = extract_num(df[e], "price")

In [16]:
# Calculate days since posting
df["days_online"] = (df["days_online"]
                     .apply(convert_elapsed_time))

In [17]:
# Convert to binary
# Current yes/no columns
binary = ["vve_kvk", "vve_am", "vve_reserve_fund", 
          "vve_maintenance", "vve_insurance"]

# Fill NaN with 0
df[["vve_per_contr"] + binary] = (df[["vve_per_contr"] + binary].fillna(0))

# Straight forward columns
for col in binary:
    df[col] = np.where(df[col] == "Ja", 1, 0)

# Columns containing numeric values as well
df["vve_per_contr"] = np.where(df["vve_per_contr"]
                               .str.contains("ja", case=False), 
                               1, 0)

# Other binary oppositions
df["new_build"] = np.where(df["new_build"] == "Nieuwbouw", 
                           1, 0)

In [18]:
# Investigate build year and/or era
df[df["build_year"].isna() & df["build_era"].notna()]

,address,postcode,city,asking_price,price_m2,days_online,service_fees_pm,vve_contribution,property_type,new_build,build_year,roof_type,specials,apartment_type,build_era,property_m2,property_m3,num_rooms,num_bathrooms,bathroom_features,floors,features,building_orientation,energy_label,isolation,heating,hot_water,boiler,energy_label_temp,garden,balcony,environment,garden_front,garden_orientation,terrace,garden_back,garden_patio,garage_type,storage_type,storage_features,storage_isolation,parking,vve_kvk,vve_am,vve_per_contr,vve_reserve_fund,vve_maintenance,vve_insurance
30,Hagedoornplein 24,1031 BV,Amsterdam,935000,8423,4,0,200.0,NaN,0,NaN,Samengesteld dak bedekt met pannen,Monumentaal pand,Maisonnette (appartement),1906-1930,NaN,438 m³,5 kamers (4 slaapkamers),2 badkamers en 2 aparte toiletten,2 ligbaden,3 woonlagen,"Dakraam, mechanische ventilatie en TV kabel",3e woonlaag,D,Volledig geïsoleerd,Cv-ketel,Cv-ketel,"Combiketel uit 2015, eigendom",NaN,NaN,NaN,Aan park en in woonwijk,NaN,NaN,NaN,NaN,NaN,NaN,Inpandig,NaN,NaN,Parkeervergunningen,1,1,1,1,1,1
43,Nieuwendammerdijk 27,1025 LC,Amsterdam,849000,6481,80,0,0.0,"Eengezinswoning, tussenwoning (dijkwoning)",0,NaN,Zadeldak bedekt met pannen,NaN,NaN,1981-1990,80 m²,479 m³,5 kamers (4 slaapkamers),3 badkamers en 2 aparte toiletten,"Ligbad, 2 douches en toilet",2 woonlagen en een kelder,Mechanische ventilatie en natuurlijke ventilatie,NaN,A,"Dubbel glas, HR-glas en volledig geïsoleerd",Cv-ketel,Cv-ketel,"Gas gestookt combiketel, eigendom",NaN,Achtertuin,Dakterras aanwezig,Aan rustige weg en in woonwijk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Betaald parkeren en parkeervergunningen,0,0,1,0,0,0
72,Saxenburgerstraat 6 II-III,1054 KP,Amsterdam,999000,7511,10,0,101.0,NaN,0,NaN,NaN,NaN,Bovenwoning (dubbel bovenhuis),1906-1930,NaN,464 m³,4 kamers (3 slaapkamers),1 badkamer en 2 aparte toiletten,Ligbad,2 woonlagen,"Buitenzonwering, dakraam, mechanische ventilat...",2e woonlaag,C,"Dakisolatie, dubbel glas, muurisolatie en vloe...",Cv-ketel,Cv-ketel,"CR Remeha (gas gestookt combiketel uit 2015, e...",NaN,NaN,Balkon aanwezig,"Aan park, aan rustige weg, in centrum en in wo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Betaald parkeren en parkeervergunningen,1,1,1,1,0,1
97,Waterloop 548,1051 PS,Amsterdam,585000,5735,14,0,154.0,NaN,0,NaN,Plat dak bedekt met bitumineuze dakbedekking,NaN,Maisonnette (appartement),1991-2000,NaN,330 m³,4 kamers (3 slaapkamers),1 badkamer en 1 apart toilet,Ligbad,4 woonlagen,Mechanische ventilatie en TV kabel,2e woonlaag,B,Volledig geïsoleerd,Stadsverwarming,Stadsverwarming,NaN,NaN,NaN,Dakterras aanwezig,"Aan rustige weg, beschutte ligging en in woonwijk",NaN,NaN,NaN,NaN,NaN,NaN,Box,NaN,NaN,Betaald parkeren,1,1,1,1,1,1
114,Tweede Keucheniusstraat 19 H,1051 VP,Amsterdam,329000,5772,14,0,110.0,NaN,0,NaN,Plat dak,NaN,Benedenwoning (appartement),1931-1944,NaN,225 m³,3 kamers (2 slaapkamers),1 badkamer en 1 apart toilet,NaN,1 woonlaag,Alarminstallatie en mechanische ventilatie,Begane grond,B,Dubbel glas,Cv-ketel,Cv-ketel,"Remeha ( combiketel uit 2011, eigendom)",NaN,Achtertuin en voortuin,NaN,"Aan rustige weg, beschutte ligging en in woonwijk",NaN,Gelegen op het zuiden,NaN,"22 m² (4 meter diep en 5,4 meter breed)",NaN,NaN,NaN,NaN,NaN,Betaald parkeren en parkeervergunningen,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3920,Brigantijnkade 36,1086 VB,Amsterdam,585000,4062,160,0,0.0,"Villa, vrijstaande woning (waterwoning)",0,NaN,Plat dak,Gemeubileerd,NaN,2001-2010,144 m²,445 m³,5 kamers (3 slaapkamers),NaN,NaN,1 woonlaag,"Buitenzonwering, jacuzzi, rolluiken en TV kabel",NaN,NaN,NaN,NaN,NaN,NaN,A,Zonneterras,NaN,"Aan vaarwater, aan water en in woonwijk",NaN,Gelegen op het zuidwesten,"0,06 meter diep en 0,06 meter breed",NaN,NaN,Garagebox,NaN,NaN,NaN,NaN,0,0,1,0,0,0
3921,Crynssenstraat 48 1,1058 XX,Amsterdam,299000,5750,49,0,0.0

In [19]:
# Calculate mean of build period
df["build_era"] = (df["build_era"]
                   .apply(build_era)
                   .astype(int, errors="ignore"))

In [20]:
# Use mean of build period if build year is null
df["build_year"] = np.where(df["build_year"].notnull(), 
                            df["build_year"], 
                            df["build_era"])
df.drop(columns=["build_era"], inplace=True)

In [21]:
# Extract number from build year
df["build_year"] = extract_num(df["build_year"], "year")

In [22]:
# Extract numbers from area/volume
for m in ["property_m2", "property_m3"]:
    df[m] = extract_num(df[m], "meter")

In [23]:
listing_type(df)

In [24]:
roof_description(df["roof_type"].copy())

In [25]:
df.head()

,address,postcode,city,asking_price,price_m2,days_online,service_fees_pm,vve_contribution,property_type,new_build,build_year,roof_type,specials,property_m2,property_m3,num_rooms,num_bathrooms,bathroom_features,floors,features,building_orientation,energy_label,isolation,heating,hot_water,boiler,energy_label_temp,garden,balcony,environment,garden_front,garden_orientation,terrace,garden_back,garden_patio,garage_type,storage_type,storage_features,storage_isolation,parking,vve_kvk,vve_am,vve_per_contr,vve_reserve_fund,vve_maintenance,vve_insurance,pt_hofjeswoning,pt_drive-in_woning,pt_portiekwoning,pt_tussenwoning,pt_benedenwoning,pt_penthouse,pt_2-onder-1-kapwoning,pt_appartement,pt_corridorflat,pt_herenhuis,pt_villa,pt_landhuis,pt_eengezinswoning,pt_bungalow,pt_hoekwoning,pt_waterwoning
0,Buiksloterbreek 41,1034 XC,Amsterdam,475000,5220,3,0,0.0,"Eengezinswoning, geschakelde woning",0,1994,Plat dak,NaN,51,379,4 kamers (3 slaapkamers),1 badkamer en 1 apart toilet,Douche en toilet,2 woonlagen,"Buitenzonwering, schuifpui en TV kabel",NaN,B,Volledig geïsoleerd,Cv-ketel,Cv-ketel,Intergas HRE (gas gestookt combiketel uit 2010...,NaN,Zonneterras,Dakterras aanwezig,NaN,NaN,NaN,NaN,NaN,NaN,Inpandig,NaN,NaN,NaN,"Betaald parkeren, openbaar parkeren en parkeer...",0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,Arènpalmstraat 16,1104 DB,Amsterdam,425000,3400,12,0,0.0,"Eengezinswoning, hoekwoning",0,2008,Plat dak,NaN,98,429,6 kamers (4 slaapkamers),1 badkamer en 1 apart toilet,"Ligbad, douche en toilet",3 woonlagen,Mechanische ventilatie en TV kabel,NaN,A,Volledig geïsoleerd,Cv-ketel en gedeeltelijke vloerverwarming,Cv-ketel,Gas gestookt combiketel,NaN,Voortuin,Dakterras aanwezig en balkon aanwezig,"Aan rustige weg, aan water, in woonwijk en vri...","36 m² (7 meter diep en 5,09 meter breed)",Gelegen op het zuidoosten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Op eigen terrein en openbaar parkeren,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
2,Brigantijnkade 46,1086 VB,Amsterdam,625000,4340,11,14,0.0,"Villa, vrijstaande woning (waterwoning)",0,2009,Plat dak bedekt met bitumineuze dakbedekking,Gedeeltelijk gestoffeerd,133,519,5 kamers (4 slaapkamers),2 badkamers,"Ligbad, 2 douches en 2 toiletten",3 woonlagen,Schuifpui,NaN,A,HR-glas en volledig geïsoleerd,Cv-ketel en gedeeltelijke vloerverwarming,Cv-ketel en elektrische boiler,"Nefit (gas gestookt combiketel uit 2017, eigen...",NaN,Zonneterras,Dakterras aanwezig en frans balkon aanwezig,"Aan rustige weg, aan vaarwater, aan water, in ...",NaN,Gelegen op het zuidwesten,"33 m² (9,4 meter diep en 3,51 meter breed)",NaN,NaN,Parkeerkelder en parkeerplaats,Inpandig,Elektra en stromend water,NaN,Betaald parkeren,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,Bosplaat 25,1025 AR,Amsterdam,700000,5833,10,0,0.0,"Bungalow, tussenwoning (semi-bungalow)",0,1973,Plat dak,NaN,160,273,4 kamers (3 slaapkamers),1 badkamer en 1 apart toilet,Toilet,2 woonlagen,Dakraam en schuifpui,NaN,C,Dubbel glas,Cv-ketel,Elektrische boiler,"Atag (gas gestookt combiketel uit 2010, eigendom)",NaN,Achtertuin en voortuin,Balkon aanwezig,Aan rustige weg en in woonwijk,NaN,Gelegen op het zuidwesten bereikbaar via achterom,NaN,"104 m² (16,71 meter diep en 6,21 meter breed)",NaN,NaN,Aangebouwde stenen berging,Elektra,NaN,"Betaald parkeren, openbaar parkeren en parkeer...",0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,Ben van Meerendonkstraat 156,1087 LN,Amsterdam,779000,4839,11,43,0.0,"Herenhuis, tussenwoning",0,2010,Plat dak bedekt met bitumineuze dakbedekking,Gedeeltelijk gestoffeerd,142,580,5 kamers (4 slaapkamers),2 badkamers en 1 apart toilet,"Ligbad, 2 douches en 2 toiletten",3 woonlagen,Schuifpui en TV kabel,NaN,A,Dubbel glas en volledig geïsoleerd,Stadsverwarming en gehele vloerverwarming,Stadsverwarming,NaN,NaN,Achtertuin,NaN,"Aan rustige weg, aan vaarwater, aan water, in ...",NaN,Gelegen op het zuidwesten,NaN,"78 m² (13,53 meter diep en 5,74 meter breed)",NaN,Parkeerkelder en parkeerplaats,Box,NaN,NaN,Betaald parkeren,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0

In [ ]:
a.tail()

In [ ]:
df.sample(20)
#df[df["property_area"].notna()]